In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

import os
import glob
import random
import shutil
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import clear_output
import matplotlib.pyplot as plt

import torchvision
from torchvision import transforms

from captum.attr import IntegratedGradients
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

In [ ]:
idx_to_labels = {'0':'control', '1':'diabetic'}
transform = transforms.Compose([
 transforms.Resize((300, 300)),
 transforms.ToTensor()
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
clear_output(wait=True)

pathModel = "models/densenet121_prep_e50_s300_b14.pt"
transform_normalize = transforms.Normalize(
     mean=[0.5023, 0.5017, 0.5019],
     std=[0.1245, 0.0934, 0.0581]
 )

model = torchvision.models.densenet121(pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))
model = model.cuda()
model.load_state_dict(torch.load(pathModel))
model = model.eval()

folder = 'heatmap_captum/prep'
if os.path.exists(folder):
    shutil.rmtree(folder)
os.makedirs(folder + '/integrated_gradients')
os.makedirs(folder + '/occlusion')

list_path = glob.glob('data/data_m_prep/test/retina/*')+glob.glob('data/data_m_prep/test/control/*')

for img_path in list_path:
    img = Image.open(img_path)
    transformed_img = transform(img)
    input = transform_normalize(transformed_img).unsqueeze(0).to(device)

    output = model(input)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)

    pred_label_idx.squeeze_()
    predicted_label = idx_to_labels[str(pred_label_idx.item())]
    if img_path.split('\\')[-1].split('.')[0].split('_')[1]=='c' or img_path.split('\\')[-1].split('.')[0].split('_')[-1] in ['h', 'N']:
        true_leb = 'control'
    else:
        true_leb = 'diabetic'
    title = 'Real: ' + true_leb + \
            '\nPredicted: ' + predicted_label + '(' + str(round(prediction_score.squeeze().item(), 4)) + ')'
    torch.cuda.empty_cache()

    integrated_gradients = IntegratedGradients(model)
    attributions_ig = integrated_gradients.attribute(input, target=pred_label_idx, n_steps=10)

    default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                     [(0, '#ffffff'),
                                                      (0.25, '#000000'),
                                                      (1, '#000000')], N=256)

    a = viz.visualize_image_attr_multiple(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 ["original_image", "heat_map", "heat_map", "masked_image"],
                                 ["all", "positive", "negative", "positive" if pred_label_idx.item() == 0 else 'negative'],
                                 show_colorbar=True,
                                 cmap = default_cmap,
                                 titles=['orginal', 'positive attribution', 'negative attribution', 'masked'],
                                 outlier_perc=1,
                                 fig_size=(30,11)
                                 )
    a[0].suptitle(title, size=30)
    a[0].savefig(folder + '/integrated_gradients/' + img_path.split('\\')[-1].split('.')[0])
    plt.close()
    
    noise_tunnel = NoiseTunnel(integrated_gradients)

    occlusion = Occlusion(model)

    attributions_occ = occlusion.attribute(input,
                                           strides = (3, 8, 8),
                                           target=pred_label_idx,
                                           sliding_window_shapes=(3,15, 15),
                                           baselines=0)

    b = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          ["original_image", "blended_heat_map", "blended_heat_map", "masked_image"],
                                          ["all", "positive", "negative", "positive" if pred_label_idx.item() == 0 else 'negative'],
                                          show_colorbar=True,
                                          titles=['orginal', 'positive attribution', 'negative attribution', 'masked'],
                                          outlier_perc=2,
                                          fig_size=(30,11)
                                         )
    b[0].suptitle(title, size=30)
    b[0].savefig(folder + '/occlusion/' + img_path.split('\\')[-1].split('.')[0])

In [ ]:
clear_output(wait=True)

pathModel = "models/densenet121_e50_s300_b14.pt"
transform_normalize = transforms.Normalize(
     mean=[0.3998, 0.1676, 0.0636],
     std=[0.2762, 0.1356, 0.0666]
 )


model = torchvision.models.densenet121(pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))
model = model.cuda()
model.load_state_dict(torch.load(pathModel))
model = model.eval()

folder = 'heatmap_captum/raw'
if os.path.exists(folder):
    shutil.rmtree(folder)
os.makedirs(folder + '/integrated_gradients')
os.makedirs(folder + '/occlusion')

list_path = glob.glob('data/data_m/test/retina/*')+glob.glob('data/data_m/test/control/*')

for img_path in list_path:
    img = Image.open(img_path)
    transformed_img = transform(img)
    input = transform_normalize(transformed_img).unsqueeze(0).to(device)

    output = model(input)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)

    pred_label_idx.squeeze_()
    predicted_label = idx_to_labels[str(pred_label_idx.item())]
    if img_path.split('\\')[-1].split('.')[0].split('_')[1]=='c' or img_path.split('\\')[-1].split('.')[0].split('_')[-1] in ['h', 'N']:
        true_leb = 'control'
    else:
        true_leb = 'diabetic'
    title = 'Real: ' + true_leb + \
            '\nPredicted: ' + predicted_label + '(' + str(round(prediction_score.squeeze().item(), 4)) + ')'
    torch.cuda.empty_cache()

    integrated_gradients = IntegratedGradients(model)
    attributions_ig = integrated_gradients.attribute(input, target=pred_label_idx, n_steps=10)

    default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                     [(0, '#ffffff'),
                                                      (0.25, '#000000'),
                                                      (1, '#000000')], N=256)

    a = viz.visualize_image_attr_multiple(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 ["original_image", "heat_map", "heat_map", "masked_image"],
                                 ["all", "positive", "negative", "positive" if pred_label_idx.item() == 0 else 'negative'],
                                 show_colorbar=True,
                                 cmap = default_cmap,
                                 titles=['orginal', 'positive attribution', 'negative attribution', 'masked'],
                                 outlier_perc=1,
                                 fig_size=(30,11)
                                 )
    a[0].suptitle(title, size=30)
    a[0].savefig(folder + '/integrated_gradients/' + img_path.split('\\')[-1].split('.')[0])
    
    noise_tunnel = NoiseTunnel(integrated_gradients)

    occlusion = Occlusion(model)

    attributions_occ = occlusion.attribute(input,
                                           strides = (3, 8, 8),
                                           target=pred_label_idx,
                                           sliding_window_shapes=(3,15, 15),
                                           baselines=0)

    b = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          ["original_image", "blended_heat_map", "blended_heat_map", "masked_image"],
                                          ["all", "positive", "negative", "positive" if pred_label_idx.item() == 0 else 'negative'],
                                          show_colorbar=True,
                                          titles=['orginal', 'positive attribution', 'negative attribution', 'masked'],
                                          outlier_perc=2,
                                          fig_size=(30,11)
                                         )
    b[0].suptitle(title, size=30)
    b[0].savefig(folder + '/occlusion/' + img_path.split('\\')[-1].split('.')[0])

In [ ]:
clear_output(wait=True)

pathModel = "models/densenet121_vessel_e50_s300_b14.pt"

model = torchvision.models.densenet121(pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))
model = model.cuda()
model.load_state_dict(torch.load(pathModel))
model = model.eval()

folder = 'heatmap_captum/vessel'
if os.path.exists(folder):
    shutil.rmtree(folder)
os.makedirs(folder + '/integrated_gradients')
os.makedirs(folder + '/occlusion')

list_path = glob.glob('data/data_m_vessel/test/retina/*')+glob.glob('data/data_m_vessel/test/control/*')

for img_path in list_path:
    img = Image.open(img_path)
    transformed_img = transform(img)
    transformed_img = transformed_img.expand(3,*transformed_img.shape[1:])

    input = transformed_img.unsqueeze(0).to(device)

    output = model(input)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)

    pred_label_idx.squeeze_()
    predicted_label = idx_to_labels[str(pred_label_idx.item())]
    if img_path.split('\\')[-1].split('.')[0].split('_')[1]=='c' or img_path.split('\\')[-1].split('.')[0].split('_')[-1] in ['h', 'N']:
        true_leb = 'control'
    else:
        true_leb = 'diabetic'
    title = 'Real: ' + true_leb + \
            '\nPredicted: ' + predicted_label + '(' + str(round(prediction_score.squeeze().item(), 4)) + ')'
    torch.cuda.empty_cache()

    integrated_gradients = IntegratedGradients(model)
    attributions_ig = integrated_gradients.attribute(input, target=pred_label_idx, n_steps=10)

    default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                     [(0, '#ffffff'),
                                                      (0.25, '#000000'),
                                                      (1, '#000000')], N=256)

    a = viz.visualize_image_attr_multiple(np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                 ["original_image", "heat_map", "heat_map", "masked_image"],
                                 ["all", "positive", "negative", "positive" if pred_label_idx.item() == 0 else 'negative'],
                                 show_colorbar=True,
                                 titles=['orginal', 'positive attribution', 'negative attribution', 'masked'],
                                 cmap=default_cmap,
                                 outlier_perc=2,
                                 fig_size=(30,11)
                                 )
    a[0].suptitle(title, size=30)
    a[0].savefig(folder + '/integrated_gradients/' + img_path.split('\\')[-1].split('.')[0])

    noise_tunnel = NoiseTunnel(integrated_gradients)

    occlusion = Occlusion(model)

    attributions_occ = occlusion.attribute(input,
                                           strides = (3, 8, 8),
                                           target=pred_label_idx,
                                           sliding_window_shapes=(3,15, 15),
                                           baselines=0)
    b = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1,2,0)),
                                          ["original_image", "blended_heat_map", "blended_heat_map", "masked_image"],
                                          ["all", "positive", "negative", "positive" if pred_label_idx.item() == 0 else 'negative'],
                                          show_colorbar=True,
                                          titles=['orginal', 'positive attribution', 'negative attribution', 'masked'],
                                          outlier_perc=2,
                                          fig_size=(30,11)
                                         )
    b[0].suptitle(title, size=30)
    b[0].savefig(folder + '/occlusion/' + img_path.split('\\')[-1].split('.')[0])